# AI算法工程师
## 毕业项目
### 猫狗识别

本节介绍构建dataset和dataloader的过程，以及使用模型预测测试图像。

#### 1.加载数据

In [4]:
%load_ext autoreload
%autoreload 2
from dataloader import CatDogDataset

In [5]:
from torchvision import transforms

transform_train = transforms.Compose([ 
    transforms.Resize(256),                          
    transforms.RandomCrop(224),                      
    transforms.RandomHorizontalFlip(),               
    transforms.ToTensor(),                          
    transforms.Normalize((0.485, 0.456, 0.406),      
                         (0.229, 0.224, 0.225))])
transform_val = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], 
                         [0.229, 0.224, 0.225])])

##### 将train数据划分为训练集和验证集

In [14]:
from utils import imagelist
#此处使用自定义方法返回文件夹下的文件名列表
train_files = imagelist('data/train')

In [7]:
import random
random.shuffle(train_files)
train_list = train_files[:20000]
valid_list = train_files[20000:]

In [8]:
train_dataset = CatDogDataset(image_paths=train_list, transform=transform_train, mode='train' )
val_dataset = CatDogDataset(image_paths=valid_list, transform=transform_val, mode='val' )

In [9]:
from torch.utils.data import Dataset, DataLoader

batch_size = 64
train_dataLoader = DataLoader(train_dataset, batch_size=batch_size)
valid_dataLoader = DataLoader(val_dataset, batch_size=batch_size)


#### 2.测试样本
使用未经训练的模型预测，因为是二分类问题，预测值应为0.5左右。

In [10]:
from model import CustomCNN, Pre_CNN_VGG

In [13]:
image = next(iter(train_dataLoader))[0]
image = image[0].unsqueeze(0)

In [69]:
model = CustomCNN()
preds = model(image)

In [72]:
import torch.nn.functional as F
preds = F.softmax(preds, dim=1)
preds

tensor([[0.4946, 0.5054]], grad_fn=<SoftmaxBackward>)

使用预训练的VGG模型预测

In [79]:
model = Pre_CNN_VGG()

In [81]:
preds = model(image)
preds = F.softmax(preds, dim=1)
preds

tensor([[0.3549, 0.6451]], grad_fn=<SoftmaxBackward>)